In [1]:
%pip install --user sklearn
%pip install --user numpy
%pip install --user matplotlib

ERROR:root:Line magic function `%pip` not found.
ERROR:root:Line magic function `%pip` not found.
ERROR:root:Line magic function `%pip` not found.


In [13]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 

In [11]:
data_matrix = np.loadtxt(open("./msd_genre_dataset/msd_genre_dataset_segpreproc_knime.csv", "rb"), delimiter=";", skiprows=1)

print("Filas de la matriz: " + str(len(data_matrix)))
print("Columnas de la matriz: " + str(len(data_matrix[0])))

X = np.resize(data_matrix, (len(data_matrix), len(data_matrix[0])-1))
y = data_matrix[:,len(data_matrix[0])-1]


print("X shape" + str(X.shape))
print("y shape" + str(y.shape))

Filas de la matriz: 8350
Columnas de la matriz: 31
X shape(8350, 30)
y shape(8350,)


In [15]:
percentage_test = 0.2

#Porcentaje de datos dedicados a validación, relativo al número de daots de test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = percentage_test) 

array([[  1.17443000e+02,   4.00000000e+00,   5.00000000e+00, ...,
          3.91004962e+02,   1.78281997e+02,   1.00000000e+00],
       [  2.34420941e+03,   1.36365092e+03,   6.19663915e+02, ...,
         -2.08812990e-02,   6.39017805e+00,   5.21939360e+01],
       [  4.38792627e+01,   2.01850118e+01,   2.85422269e+00, ...,
          3.00000000e+00,   1.00000000e+00,   1.00000000e+00],
       ..., 
       [  3.00000000e+00,   5.00000000e+00,   1.00000000e+00, ...,
          2.83430235e+02,   1.00000000e+00,  -1.17630000e+01],
       [  2.77961891e+00,  -6.76128940e+00,   3.40002746e+01, ...,
         -2.31698757e+01,  -8.32152094e+00,  -7.74862565e+00],
       [ -1.12400586e+01,   3.05356616e+00,  -1.66671334e+01, ...,
          2.94059950e+02,   2.76560422e+01,  -8.27561972e+00]])